In [ ]:
import time
import image_similarity
model_name = r'D:\SPRING-20\696-E\image-similarity\deepranking-v2-150000.h5'
query_image = r'D:\SPRING-20\696-E\image-similarity\airplane.jpg'
img_directory = r'D:\SPRING-20\696-E\image-similarity\val_256'
K = 10
start_time = time.time()
image_similarity.loadData(model_name, img_directory, mydb)
print(time.time() - start_time)

In [ ]:
from SceneClassification import get_image_scene
from Embedding import getEmbedding
import os
import json
import heapq
import mysql.connector


def getSimilarImages(model_name, query_image_path, img_directory, mydb, K):
    
    
    try:
        scene_type = get_image_scene(query_image_path)
    except:
        scene_type = ''
    print(scene_type)
    distance = {}
    sql = "SELECT * FROM metadata where scene_type = %s"
    val = (scene_type, )
    mycursor = mydb.cursor()
    mycursor.execute(sql,val)
    resultset = mycursor.fetchall()
    embedding_dict = dict()
    isEmbeddingPresent = False
    for result in resultset:
        imagePath = result[1]
        if imagePath == query_image_path:
            isEmbeddingPresent = True
        embedStr = result[2]
        embedding_dict[imagePath] = json.loads(embedStr)
    
    if not isEmbeddingPresent:
        query_image_embedding = getEmbedding(model_name, query_image_path).tolist()
    else:
        query_image_embedding = embedding_dict[query_image_path]
        del embedding_dict[query_image_path]

    for img in embedding_dict:
        candidate_embedding = embedding_dict[img]
        distance[img] = sum([(query_image_embedding[idx] - candidate_embedding[idx])**2 for idx in range(len(query_image_embedding))])**(0.5)
    
    heap = [(value, key) for key,value in distance.items()]
    largestK = heapq.nsmallest(K, heap)
    return largestK

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  port = "8080",
  database = "image_similarity",  
  user="root",
  password="imagesim@cs696",
  auth_plugin='mysql_native_password'
    
)

model_name = r'D:\SPRING-20\696-E\image-similarity\deepranking-v2-150000.h5'
query_image = r'D:\SPRING-20\696-E\image-similarity\val_256\Places365_val_00018627.jpg'
img_directory = r'D:\SPRING-20\696-E\image-similarity\val_256'
K = 10

In [ ]:
%pylab inline
import image_similarity
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

sorted_image_scores = image_similarity.getSimilarImages(model_name, query_image, img_directory, mydb ,K)

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

print("\n"+color.BOLD + "Query image: " + color.END + "\n")

img=mpimg.imread(query_image)

imgplot = plt.imshow(img)

plt.show()

print(color.BOLD + "Results:" + color.END + "\n")

for image in sorted_image_scores:
    img=mpimg.imread(image[1])
    imgplot = plt.imshow(img)
    plt.show()